In [3]:
from google.colab import files
uploaded = files.upload()

Saving sys-log.csv to sys-log.csv


In [24]:
# CNN for the logs classificaton
import numpy
import pandas
import re
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Convolution1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.utils import np_utils
# utils functions define
def str2numbers(strs):
	res = []
	for str in strs:
		wArr = []
		#split by whitespace and /
		words = re.split("[\s\/]",str)
	  
		for word in words:

			wArr.append(sum([ord(c) for c in word]))
		res.append(wArr)
	return res

	
	
def predictions2className(classes,predictions):
	res = []
	for prediction in predictions:
		index = numpy.argmax(prediction)
		res.append(classes[index])
	return res
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
dataframe = pandas.read_csv('sys-log.csv', sep='---', engine='python')

dataset = dataframe.values
numpy.random.shuffle(dataset)
#test log length 85
X_str = dataset[:,0]

#label length 85
y_str = dataset[:,1]

train_size = int(len(X_str) * 0.8)
test_size = len(X_str) - train_size
#sum of unicode of each word
X_num = str2numbers(X_str)

X_train = X_num[:train_size]
X_test = X_num[train_size:]
#unique array, index to construct back to input array 
classes, y_num = numpy.unique(y_str, return_inverse=True)

#(85,17) 
y_oneHot = np_utils.to_categorical(y_num)

(68,17)
y_train = y_oneHot[:train_size]
#(17,17)
y_test = y_oneHot[train_size:]
# pad dataset to a maximum review length in words
max_words = 100
#(68,100)
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
#(17,100)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)
# create the model
top_words = 1000000
#classify into 17
outputs = len(classes)
"""
选择模型
"""
model = Sequential()
"""
构建网络层
"""
#词嵌入 
model.add(Embedding(top_words, 32, input_length=max_words))

model.add(Convolution1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(outputs, activation='softmax'))

"""
编译
"""
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())


# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=30, batch_size=1, verbose=1)
# Final evaluation of the model

'''
输出
'''
#loss and accuracy
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
#(17,17)
predictions = model.predict(X_test, verbose=0)

result = predictions2className(classes, predictions)
print ("############## Test logs: ",X_str[train_size:])
print ("############## Log types: ",y_str[train_size:])
print ("############## Prediction types: ",result)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 32)           32000000  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 100, 32)           3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 50, 32)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 250)               400250    
_________________________________________________________________
dense_3 (Dense)              (None, 17)                4267      
Total params: 32,407,621
Trainable params: 32,407,621
Non-trainable params: 0
__________________________________________

In [31]:
# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/new_model') 

INFO:tensorflow:Assets written to: saved_model/new_model/assets


In [26]:
# my_model directory
!ls saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
!ls saved_model/new_model

my_model
assets	saved_model.pb	variables


In [32]:

import tensorflow as tf
new_model = tf.keras.models.load_model('saved_model/new_model')

# Check its architecture
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 32)           32000000  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 100, 32)           3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 50, 32)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 250)               400250    
_________________________________________________________________
dense_3 (Dense)              (None, 17)                4267      
Total params: 32,407,621
Trainable params: 32,407,621
Non-trainable params: 0
__________________________________________

In [39]:
# Evaluate the restored model
loss, acc = new_model.evaluate(X_test, y_test,verbose=0)

print("Accuracy: %.2f%%" % (acc*100))

print(new_model.predict(X_test).shape)

Accuracy: 94.12%
(17, 17)
